## Funnel and Cohort Analysis

**Background**

Pada data Funnel - cohort web app,  ada dua sheet event dan sales, event merupakan table record mengenai journey user pada web/aplikasi kita, maka untuk menentukan traffic funnelnya perlu menghitung berapa banyak unique session id di dalamnya. 

Untuk menjawab no.3 di mana membutuhkan dua gabungan table yaitu event dan sales, created at merujuk tanggal user visit pada web/app maka untuk menghitung churn rate diperlukan cohort dengan menggunakan data event.

Untuk menjawab mengenai repurchase, teman teman diminta untuk dapat menggabungkan kedua table tersebut dengan ID yang tersedia, untuk melihat record purchase activity dari user yang berkunjung.

## 1. Membuat Dataset untuk Funnel & Cohort Analysis

- Siapkan dataset yang sesuai untuk masing-masing analisis.
- Pisahkan dataset yang akan digunakan untuk Funnel Analysis dan Cohort Analysis.
- Pastikan dataset memiliki event timestamp, user ID, dan event type yang relevan.
- Lakukan cleaning data jika diperlukan, seperti menghapus duplikasi atau menangani missing values.
Goals: Menyusun dataset yang siap digunakan untuk analisis funnel dan cohort.

In [1]:
import pandas as pd

In [2]:
df_event = pd.read_excel('Assignment Datasets Funnel Cohort.xlsx', sheet_name='Events')
df_sales = pd.read_excel('Assignment Datasets Funnel Cohort.xlsx', sheet_name='Sales')

In [3]:
df_event.head()

,id,user_id,age,gender,city,country,session_id,sequence_number,created_at,browser,traffic_source,event_type
0,555311,42922.0,46.0,M,Bogatynia,Poland,f65f7f3f-1078-45da-a7af-2dd41a894e5d,3,2023-01-19 22:54:38 UTC,Other,Email,cart
1,714816,55343.0,16.0,M,Bogatynia,Poland,cc3509fc-641b-4632-aca7-4949209b9afb,4,2023-11-26 17:12:19 UTC,Safari,Email,cart
2,464405,35775.0,46.0,F,Bogatynia,Poland,16f8b065-9b2e-47bf-b8e7-793060cfbe04,9,2023-09-27 02:46:04 UTC,Firefox,Adwords,cart
3,649908,50243.0,47.0,M,Bogatynia,Poland,8b145bdf-4ddd-473a-8095-c35504afa112,3,2023-04-16 08:50:42 UTC,Chrome,Adwords,cart
4,874098,67437.0,30.0,M,Zgorzelec,Poland,4871ce1f-0b5c-446d-aa7c-016c959cbe56,3,2023-05-18 04:24:49 UTC,Firefox,Email,cart


In [4]:
df_sales.head()

,order_items_id,order_id,order_date,user_id,age,gender,product_id,product_name,product_category,product_price,product_cost,item_qty
0,7005,4790,2023-06-09,3802,18,F,8033,Embroidered Capri Set - Sizes: SMMEDLG,Clothing Sets,19.99,11.49,1
1,55182,38050,2023-03-17,30411,37,F,8038,Embroidered Capri Set - Sizes: 1X2X3X4X,Clothing Sets,22.99,13.86,1
2,107743,74517,2023-11-19,59668,28,F,8038,Embroidered Capri Set - Sizes: 1X2X3X4X,Clothing Sets,22.99,13.86,1
3,8139,5591,2023-05-13,4410,64,F,8028,Collections Etc - Autumn Floral V Neck Top And...,Clothing Sets,32.97,20.80,1
4,149787,103464,2023-05-04,82891,37,F,8028,Collections Etc - Autumn Floral V Neck Top And...,Clothing Sets,32.97,20.80,1


In [5]:
df_event.columns

Index(['id', 'user_id', 'age', 'gender', 'city', 'country', 'session_id',
       'sequence_number', 'created_at', 'browser', 'traffic_source',
       'event_type'],
      dtype='object')

In [6]:
df_sales.columns

Index(['order_items_id', 'order_id', 'order_date', 'user_id', 'age', 'gender',
       'product_id', 'product_name', 'product_category', 'product_price',
       'product_cost', 'item_qty'],
      dtype='object')

In [7]:
df_event['user_id'].value_counts()

user_id
38597.0    86
87946.0    86
96291.0    82
12156.0    77
73636.0    76
           ..
29818.0     1
94181.0     1
2173.0      1
75939.0     1
99195.0     1
Name: count, Length: 28969, dtype: int64

In [8]:
df_event['event_type'].value_counts()

event_type
product     182825
cart        139267
purchase     50185
home         24234
cancel       21768
Name: count, dtype: int64

In [9]:
df_event['session_id'].value_counts()

session_id
1875d68c-71db-45ce-ae56-f6add2524e4c    9
16f8b065-9b2e-47bf-b8e7-793060cfbe04    9
4871ce1f-0b5c-446d-aa7c-016c959cbe56    9
8230acd8-95b3-4a01-aaf6-dfccc4271309    9
7da45045-ac38-4138-afd6-3ee59941c163    9
                                       ..
506afa86-51b2-48ec-b08a-bccc25106611    1
a2be6c37-9aad-415a-8e52-ac95ae091f68    1
5351e652-e996-4409-afe8-f6179bafc474    1
2a32e0ff-632f-483b-964d-d6da5f60e9ee    1
55694313-123c-4fb5-99da-a87df68cc8ae    1
Name: count, Length: 137540, dtype: int64

#### Data Cleaning

In [10]:
# ========== 1. DATA CLEANING ==========
# Drop rows with missing essential fields
# df_event = df_event.dropna(subset=['user_id', 'session_id', 'event_type', 'created_at'])
# df_sales = df_sales.dropna(subset=['user_id', 'order_id', 'order_date'])

# Ensure datetime format
# df_event['created_at'] = pd.to_datetime(df_event['created_at'], errors='coerce')
# df_sales['order_date'] = pd.to_datetime(df_sales['order_date'], errors='coerce')

In [11]:
df_event.shape

(418279, 12)

In [12]:
df_event['user_id'].isna().value_counts()

user_id
False    265687
True     152592
Name: count, dtype: int64

In [13]:
print(df_sales['user_id'].isna().value_counts())
print(df_sales['order_id'].isna().value_counts())
print(df_sales['order_date'].isna().value_counts())

user_id
False    37354
Name: count, dtype: int64
order_id
False    37354
Name: count, dtype: int64
order_date
False    37354
Name: count, dtype: int64


In [14]:
print(df_event['session_id'].isna().value_counts())
print(df_event['event_type'].isna().value_counts())
print(df_event['created_at'].isna().value_counts())

session_id
False    418279
Name: count, dtype: int64
event_type
False    418279
Name: count, dtype: int64
created_at
False    418279
Name: count, dtype: int64


### 1.1 Data Event version 1 for Funnel

In [15]:
# Bersihkan & proses tanggal
df_event['created_at'] = pd.to_datetime(df_event['created_at'], errors='coerce')
df_event['date'] = df_event['created_at'].dt.date
# print(df_event['created_at'].isna().sum())
# print(df_event['date'].unique())
# Hitung jumlah session unik per step (event_type)
df_event_clean = df_event.drop_duplicates(subset=['session_id', 'event_type']) #clean again for real...
funnel_summary = df_event_clean.groupby(['date', 'event_type']).session_id.nunique().reset_index()
funnel_summary = funnel_summary.rename(columns={'session_id': 'total_session'})

# Optional: rapikan nama action (untuk visual Power BI)
funnel_summary['event_type'] = funnel_summary['event_type'].str.replace('_', ' ').str.title()#.reset_index()

# Simpan ke Excel
# funnel_summary.to_excel('funnel_summary.xlsx', index=False)
# # Simpan ke CSV
funnel_summary.to_csv('funnel_summary.csv', index=False)

In [16]:
df_event['created_at'] = pd.to_datetime(
    df_event['created_at'], 
    format='%Y-%m-%d %H:%M:%S',  # Ganti dengan format sesuai data
    errors='coerce'
)
df_event['created_at'].head()

0   2023-01-19 22:54:38+00:00
1   2023-11-26 17:12:19+00:00
2   2023-09-27 02:46:04+00:00
3   2023-04-16 08:50:42+00:00
4   2023-05-18 04:24:49+00:00
Name: created_at, dtype: datetime64[ns, UTC]

In [17]:
print(funnel_summary)

            date event_type  total_session
0     2023-01-01     Cancel             62
1     2023-01-01       Cart            219
2     2023-01-01       Home             43
3     2023-01-01    Product            342
4     2023-01-01   Purchase             96
...          ...        ...            ...
1820  2023-12-31     Cancel             60
1821  2023-12-31       Cart            291
1822  2023-12-31       Home             88
1823  2023-12-31    Product            409
1824  2023-12-31   Purchase            180

[1825 rows x 3 columns]


In [18]:
# Definisikan urutan funnel yang benar
funnel_order = {
    'home': 1,
    'product': 2,
    'cart': 3,
    'purchase': 4
}
funnel_sequence = funnel_summary.copy()

# Tambahkan kolom step_order berdasarkan mapping
funnel_sequence['step_order'] = funnel_sequence['event_type'].str.lower().map(funnel_order)

# Filter hanya langkah funnel utama (jika ingin)
funnel_sequence = funnel_sequence[funnel_sequence['step_order'].notnull()]

# Urutkan funnel untuk Power BI
funnel_sequence = funnel_sequence.sort_values(by=['date', 'step_order']).reset_index()
funnel_sequence.drop('index', axis=1, inplace=True)

# Simpan ulang untuk Power BI
funnel_sequence.to_csv('funnel_sequence.csv', index=False)

In [19]:
funnel_sequence.head(10)

,date,event_type,total_session,step_order
0,2023-01-01,Home,43,1.0
1,2023-01-01,Product,342,2.0
2,2023-01-01,Cart,219,3.0
3,2023-01-01,Purchase,96,4.0
4,2023-01-02,Home,48,1.0
5,2023-01-02,Product,374,2.0
6,2023-01-02,Cart,230,3.0
7,2023-01-02,Purchase,102,4.0
8,2023-01-03,Home,56,1.0
9,2023-01-03,Product,359,2.0


### 1.2. Data Event version 2 for Funnel

In [20]:
# df_event = pd.read_excel('Assignment Datasets Funnel Cohort.xlsx', sheet_name='Events')
df_event['created_at'] = pd.to_datetime(df_event['created_at'], errors='coerce')
df_event = df_event.sort_values(['user_id', 'session_id', 'sequence_number'])

# Summary: langkah-langkah funnel per sesi
funnel_summary_ = df_event.groupby(['user_id', 'session_id']) \
    .agg({
        'sequence_number': 'max',
        'event_type': lambda x: ' → '.join(x),
        'created_at': ['min', 'max'],
        'browser': 'first',
        'gender': 'first',
        'age': 'first',
        'city': 'first',
        'traffic_source': 'first'
    }).reset_index()

# Rename multiindex columns
funnel_summary_.columns = ['user_id', 'session_id', 'max_sequence', 'funnel_path',
                          'start_time', 'end_time', 'browser', 'gender', 'age', 'city', 'traffic_source']
funnel_summary_.to_csv('funnel_summary_.csv', index=False)

In [21]:
# Pisahkan setiap step funnel menjadi baris baru
df_funnel_steps = funnel_summary_.copy()  # asumsi dataframe kamu bernama df_funnel
df_funnel_steps['funnel_step_list'] = df_funnel_steps['funnel_path'].str.split(' → ')

# Explode jadi baris per step
df_funnel_steps = df_funnel_steps.explode('funnel_step_list')

# Hitung total session per step
funnel_step_counts = df_funnel_steps.groupby('funnel_step_list')['session_id'].nunique().reset_index()
funnel_step_counts.columns = ['funnel_step', 'total_sessions']

# Simpan ke Excel untuk Power BI
funnel_step_counts.to_excel('funnel_step_counts.xlsx', index=False)
funnel_step_counts.to_csv('funnel_step_counts.csv', index=False)
funnel_step_counts.head()

,funnel_step,total_sessions
0,cart,50243
1,home,24234
2,product,50243
3,purchase,50185


In [22]:
funnel_gender = df_funnel_steps.groupby(['gender', 'funnel_step_list'])['session_id'].nunique().reset_index()
funnel_gender.to_csv('funnel_gender.csv', index=False)
funnel_gender.head()

,gender,funnel_step_list,session_id
0,F,cart,25438
1,F,home,12182
2,F,product,25438
3,F,purchase,25402
4,M,cart,24805


In [23]:
funnel_step_counts

,funnel_step,total_sessions
0,cart,50243
1,home,24234
2,product,50243
3,purchase,50185


## 2. Dataset for Cohort Analysis

In [24]:
# Pastikan format datetime
df_event['created_at'] = pd.to_datetime(df_event['created_at'], errors='coerce')
df_event['created_at_date'] = df_event['created_at'].dt.normalize()

# Hitung cohort date (tanggal kunjungan pertama) per user
df_event['cohort_date'] = df_event.groupby('user_id')['created_at_date'].transform('min')

# Hitung selisih hari dari cohort (cohort_index)
df_event['cohort_index'] = (df_event['created_at_date'] - df_event['cohort_date']).dt.days

# Pilih kolom relevan dan drop duplikasi
user_cohort_df = df_event[['user_id', 'cohort_date', 'created_at_date', 'cohort_index']].drop_duplicates()

# Rename kolom biar lebih Power BI friendly
user_cohort_df.columns = ['user_id', 'cohort_date', 'visit_date', 'cohort_day']

# Simpan ke CSV
user_cohort_df.to_csv('user_cohort_detail.csv', index=False)
user_cohort_df.head()

,user_id,cohort_date,visit_date,cohort_day
211247,2.0,2023-07-04 00:00:00+00:00,2023-07-04 00:00:00+00:00,0.0
405602,3.0,2023-07-30 00:00:00+00:00,2023-07-30 00:00:00+00:00,0.0
238448,16.0,2023-03-14 00:00:00+00:00,2023-03-14 00:00:00+00:00,0.0
394814,18.0,2023-03-09 00:00:00+00:00,2023-03-09 00:00:00+00:00,0.0
367245,18.0,2023-03-09 00:00:00+00:00,2023-03-13 00:00:00+00:00,4.0


In [25]:
user_cohort_df = pd.read_csv('user_cohort_detail.csv', parse_dates=['cohort_date', 'visit_date'])

# Filter rows where cohort_date ≠ visit_date (i.e., repeat visits)
repeat_visits = user_cohort_df[user_cohort_df['cohort_date'] != user_cohort_df['visit_date']]

# Sort for better readability (optional)
repeat_visits_sorted = repeat_visits.sort_values(by=['user_id', 'cohort_date', 'visit_date'])

# Show sample rows
repeat_visits_sorted.head(10)

,user_id,cohort_date,visit_date,cohort_day
5,18.0,2023-03-09 00:00:00+00:00,2023-03-12 00:00:00+00:00,3.0
4,18.0,2023-03-09 00:00:00+00:00,2023-03-13 00:00:00+00:00,4.0
7,26.0,2023-07-02 00:00:00+00:00,2023-09-16 00:00:00+00:00,76.0
10,27.0,2023-05-15 00:00:00+00:00,2023-05-16 00:00:00+00:00,1.0
11,27.0,2023-05-15 00:00:00+00:00,2023-05-17 00:00:00+00:00,2.0
16,57.0,2023-02-04 00:00:00+00:00,2023-02-08 00:00:00+00:00,4.0
19,58.0,2023-07-15 00:00:00+00:00,2023-07-17 00:00:00+00:00,2.0
18,58.0,2023-07-15 00:00:00+00:00,2023-07-18 00:00:00+00:00,3.0
21,63.0,2023-03-19 00:00:00+00:00,2023-03-23 00:00:00+00:00,4.0
27,70.0,2023-07-01 00:00:00+00:00,2023-07-02 00:00:00+00:00,1.0


#### **cohort day = 0**

In [26]:
cohort_df = user_cohort_df

In [27]:
cohort_day_0 = cohort_df[cohort_df['cohort_day'] == 0]
cohort_day_0.sort_values(by='user_id', ascending=True).tail(10)

,user_id,cohort_date,visit_date,cohort_day
52798,99979.0,2023-07-31 00:00:00+00:00,2023-07-31 00:00:00+00:00,0.0
52801,99983.0,2023-12-08 00:00:00+00:00,2023-12-08 00:00:00+00:00,0.0
52802,99985.0,2023-04-13 00:00:00+00:00,2023-04-13 00:00:00+00:00,0.0
52805,99987.0,2023-10-10 00:00:00+00:00,2023-10-10 00:00:00+00:00,0.0
52806,99989.0,2023-01-22 00:00:00+00:00,2023-01-22 00:00:00+00:00,0.0
52807,99991.0,2023-05-02 00:00:00+00:00,2023-05-02 00:00:00+00:00,0.0
52808,99992.0,2023-06-26 00:00:00+00:00,2023-06-26 00:00:00+00:00,0.0
52812,99997.0,2023-08-07 00:00:00+00:00,2023-08-07 00:00:00+00:00,0.0
52815,99998.0,2023-05-25 00:00:00+00:00,2023-05-25 00:00:00+00:00,0.0
52816,99999.0,2023-02-19 00:00:00+00:00,2023-02-19 00:00:00+00:00,0.0


#### **filtering active users**

In [28]:
cohort_df_filtered = cohort_df[cohort_df['user_id'] > 1]
cohort_df_filtered

,user_id,cohort_date,visit_date,cohort_day
0,2.0,2023-07-04 00:00:00+00:00,2023-07-04 00:00:00+00:00,0.0
1,3.0,2023-07-30 00:00:00+00:00,2023-07-30 00:00:00+00:00,0.0
2,16.0,2023-03-14 00:00:00+00:00,2023-03-14 00:00:00+00:00,0.0
3,18.0,2023-03-09 00:00:00+00:00,2023-03-09 00:00:00+00:00,0.0
4,18.0,2023-03-09 00:00:00+00:00,2023-03-13 00:00:00+00:00,4.0
...,...,...,...,...
52812,99997.0,2023-08-07 00:00:00+00:00,2023-08-07 00:00:00+00:00,0.0
52813,99997.0,2023-08-07 00:00:00+00:00,2023-08-11 00:00:00+00:00,4.0
52814,99998.0,2023-05-25 00:00:00+00:00,2023-09-27 00:00:00+00:00,125.0
52815,99998.0,2023-05-25 00:00:00+00:00,2023-05-25 00:00:00+00:00,0.0


#### **date target = 2023-10-01**

In [29]:
df_event[df_event['cohort_date'] == '2023-10-01']['user_id'].nunique()

85

In [30]:
# Coba validasi cohort_date yang dicurigai
cohort_target = pd.to_datetime('2023-10-01')
n_user = df_event[df_event['cohort_date'] == cohort_target]['user_id'].nunique()
print(f"User dengan cohort_date 2023-10-01: {n_user}")

User dengan cohort_date 2023-10-01: 0


## 3. Dataset for Retention/Churn Analysis

In [31]:
df_sales.head()

,order_items_id,order_id,order_date,user_id,age,gender,product_id,product_name,product_category,product_price,product_cost,item_qty
0,7005,4790,2023-06-09,3802,18,F,8033,Embroidered Capri Set - Sizes: SMMEDLG,Clothing Sets,19.99,11.49,1
1,55182,38050,2023-03-17,30411,37,F,8038,Embroidered Capri Set - Sizes: 1X2X3X4X,Clothing Sets,22.99,13.86,1
2,107743,74517,2023-11-19,59668,28,F,8038,Embroidered Capri Set - Sizes: 1X2X3X4X,Clothing Sets,22.99,13.86,1
3,8139,5591,2023-05-13,4410,64,F,8028,Collections Etc - Autumn Floral V Neck Top And...,Clothing Sets,32.97,20.80,1
4,149787,103464,2023-05-04,82891,37,F,8028,Collections Etc - Autumn Floral V Neck Top And...,Clothing Sets,32.97,20.80,1


In [32]:
# df_sales = pd.read_excel('Assignment Datasets Funnel Cohort.xlsx', sheet_name='Sales')
df_sales['order_date'] = pd.to_datetime(df_sales['order_date'], errors='coerce')
df_sales['order_id'] = df_sales['order_id'].drop_duplicates()
# Ambil hanya informasi penting
df_sales_summary = df_sales.groupby('user_id').agg({
    'order_id': 'nunique',  # jumlah order
    'order_date': ['min', 'max'],  # range waktu order
    'product_price': 'sum',  # total belanja
    'product_category': lambda x: ', '.join(set(x))
}).reset_index()

# Rename
df_sales_summary.columns = ['user_id', 'total_orders', 'first_order_date', 'last_order_date',
                            'total_spent', 'purchased_categories']
df_sales_summary.head()

,user_id,total_orders,first_order_date,last_order_date,total_spent,purchased_categories
0,2,1,2023-07-04,2023-07-04,50.99,Blazers & Jackets
1,16,1,2023-03-14,2023-03-14,25.00,Tops & Tees
2,25,1,2023-08-13,2023-08-13,48.93,Plus
3,26,1,2023-09-16,2023-09-16,54.99,Fashion Hoodies & Sweatshirts
4,27,1,2023-05-16,2023-05-17,43.00,"Underwear, Tops & Tees"


In [33]:
df_sales_summary.to_csv('sales_summary.csv', index=False)

In [34]:
# Cohort analysis
# import datetime as dt  # Import modul datetime
# import matplotlib.pyplot as plt
# import seaborn as sns

# def get_month(x): 
#     return dt.datetime(x.year, x.month, 1)
    
# df_sales['cohort_month'] = df_sales['order_date'].apply(get_month)
# grouping = df_sales.groupby('user_id')['cohort_month']
# df_sales['cohort_index'] = df_sales.apply(lambda row: (row['order_date'].year - row['cohort_month'].year) * 12 
#                                    + (row['order_date'].month - row['cohort_month'].month) + 1, axis=1)

# # Calculate retention rates
# cohort_data = df_sales.groupby(['cohort_month', 'cohort_index'])['user_id'].nunique().unstack(1)
# cohort_sizes = cohort_data.iloc[:,0]
# retention = cohort_data.divide(cohort_sizes, axis=0) * 100

# # Visualize
# plt.figure(figsize=(15,8))
# plt.title('Retention Rates')
# sns.heatmap(retention, annot=True, fmt='.1f', cmap='Blues')
# plt.show()